In [1]:
import pandas as pd
import os
# from tqdm import tqdm

In [3]:
path = './data/생활이동_자치구_202309/'
file_list = os.listdir(path)

In [ ]:
data = pd.read_csv(path + file_list[0], encoding='CP949')

for file in file_list[1:]:
    tmp_data =  pd.read_csv(path + file, encoding='CP949')
    data = pd.concat([data, tmp_data], ignore_index=True)

data.head()

In [ ]:
# 출발, 도착 모두 서울인 데이터만
data = data[(data['도착 시군구 코드'] < 21000) & (data['출발 시군구 코드'] < 21000)]

# 인구수가 3명 미만이면 *
data.loc[data['이동인구(합)'] =='*', '이동인구(합)'] = 0

In [ ]:
# 타입 변경
data['대상연월'] = data['대상연월'].astype('str')
data['출발 시군구 코드'] = data['출발 시군구 코드'].astype('str')
data['도착 시군구 코드'] = data['도착 시군구 코드'].astype('str')
data['이동인구(합)'] = data['이동인구(합)'].astype('float')

In [ ]:
# 연을 월로 변경 및 타입 변경
data['대상연월'] = data['대상연월'].str[-2:]
data.rename(columns = {'대상연월' : '유동월'}, inplace = True)
data.head()

In [ ]:
# 10세 단위로 변경
data['나이'] = data['나이'] // 10 * 10
data = data.groupby(['유동월', '요일', '도착시간', '출발 시군구 코드', '도착 시군구 코드', '성별', '나이', '이동유형']).aggregate({'평균 이동 시간(분)': 'mean', '이동인구(합)': 'sum'}).reset_index()

# 20대부터 60대까지 축소
data = data[(data['나이'] >= 20) & (data['나이'] < 70)]

# 이동인구(합) INT로 변경
data['이동인구(합)'] = data['이동인구(합)'].astype('int')

data.head()

In [ ]:
# 남자는 0, 여자는 1로 변환
data['성별'] = data['성별'].apply(lambda x : 0 if x == 'F' else 1)

# 요일 영어로 변환
day_of_week = {'월' : 'MON',
               '화' : 'TUE',
               '수' : 'WED',
               '목' : 'THU',
               '금' : 'FRI',
               '토' : 'SAT',
               '일' : 'SUN',}
data['요일'] = data['요일'].apply(lambda x : day_of_week[x])

In [ ]:
# 시군구 코드 변경
sigungu = {'11010': '1111000000', '11020': '1114000000', '11030': '1117000000', 
           '11040': '1120000000', '11050': '1121500000', '11060': '1123000000', 
           '11070': '1126000000', '11080': '1129000000', '11090': '1130500000', 
           '11100': '1132000000', '11110': '1135000000', '11120': '1138000000', 
           '11130': '1141000000', '11140': '1144000000', '11150': '1147000000', 
           '11160': '1150000000', '11170': '1153000000', '11180': '1154500000', 
           '11190': '1156000000', '11200': '1159000000', '11210': '1162000000', 
           '11220': '1165000000', '11230': '1168000000', '11240': '1171000000', 
           '11250': '1174000000'}
data['출발 시군구 코드'] = data['출발 시군구 코드'].apply(lambda x : sigungu[x])
data['도착 시군구 코드'] = data['도착 시군구 코드'].apply(lambda x : sigungu[x])

In [ ]:
data = data.reset_index(drop=True)

In [ ]:
data.tail()

In [ ]:
data.info()

In [ ]:
data.to_csv(f'./data/preprocessing_data/processing_data_9.csv', encoding='utf-8', index=False)

In [ ]:
pd.read_csv('./data/preprocessing_data/processing_data_9.csv', encoding='utf-8')

# 1~9월 붙히기

In [11]:
path = './data/생활이동_자치구_20230'

for i in range(1, 10):
    # 월 별로 데이터 import
    path_month = path + str(i) + '/'
    file_list = os.listdir(path_month)
    data = pd.read_csv(path_month + file_list[0], encoding='CP949')

    # 월 모든 데이터 import 
    for file in file_list[1:]:
        tmp_data =  pd.read_csv(path_month + file, encoding='CP949')
        data = pd.concat([data, tmp_data], ignore_index=True)

    print(i)
    print(data.head(1))
    # 출발, 도착 모두 서울인 데이터만
    data = data[(data['도착 시군구 코드'] < 21000) & (data['출발 시군구 코드'] < 21000)]
    
    # 인구수가 3명 미만이면 *
    data.loc[data['이동인구(합)'] =='*', '이동인구(합)'] = 0

    # 타입 변경
    data['대상연월'] = data['대상연월'].astype('str')
    data['출발 시군구 코드'] = data['출발 시군구 코드'].astype('str')
    data['도착 시군구 코드'] = data['도착 시군구 코드'].astype('str')
    data['이동인구(합)'] = data['이동인구(합)'].astype('float')
    
    # 연을 월로 변경 및 타입 변경
    data['대상연월'] = data['대상연월'].str[-2:]
    data.rename(columns = {'대상연월' : '유동월'}, inplace = True)

    # 10세 단위로 변경
    data['나이'] = data['나이'] // 10 * 10
    data = data.groupby(['유동월', '요일', '도착시간', '출발 시군구 코드', '도착 시군구 코드', '성별', '나이', '이동유형']).aggregate({'평균 이동 시간(분)': 'mean', '이동인구(합)': 'sum'}).reset_index()
    # 20대부터 60대까지 축소
    data = data[(data['나이'] >= 20) & (data['나이'] < 70)]
    
    # 이동인구(합) INT로 변경
    data['이동인구(합)'] = data['이동인구(합)'].astype('int')

    # 남자는 0, 여자는 1로 변환
    data['성별'] = data['성별'].apply(lambda x : 0 if x == 'F' else 1)
    
    # 요일 영어로 변환
    day_of_week = {'월' : 'MON',
                   '화' : 'TUE',
                   '수' : 'WED',
                   '목' : 'THU',
                   '금' : 'FRI',
                   '토' : 'SAT',
                   '일' : 'SUN',}
    data['요일'] = data['요일'].apply(lambda x : day_of_week[x])

    # 시군구 코드 변경
    sigungu = {'11010': '1111000000', '11020': '1114000000', '11030': '1117000000', 
               '11040': '1120000000', '11050': '1121500000', '11060': '1123000000', 
               '11070': '1126000000', '11080': '1129000000', '11090': '1130500000', 
               '11100': '1132000000', '11110': '1135000000', '11120': '1138000000', 
               '11130': '1141000000', '11140': '1144000000', '11150': '1147000000', 
               '11160': '1150000000', '11170': '1153000000', '11180': '1154500000', 
               '11190': '1156000000', '11200': '1159000000', '11210': '1162000000', 
               '11220': '1165000000', '11230': '1168000000', '11240': '1171000000', 
               '11250': '1174000000'}
    data['출발 시군구 코드'] = data['출발 시군구 코드'].apply(lambda x : sigungu[x])
    data['도착 시군구 코드'] = data['도착 시군구 코드'].apply(lambda x : sigungu[x])
    
    print(data.head(1))
    # 월별로 데이터 저장
    data.to_csv(f'./data/preprocessing_data/processing_data_{i}.csv', encoding='utf-8', index=False)
    print('---------------------------------------------------------------------------------------')
    
    del data

1
     대상연월 요일  도착시간  출발 시군구 코드  도착 시군구 코드 성별  나이 이동유형  평균 이동 시간(분) 이동인구(합)
0  202301  일     0      11010      11010  F   0   EE          112   28.03
  유동월   요일  도착시간   출발 시군구 코드   도착 시군구 코드  성별  나이 이동유형  평균 이동 시간(분)  이동인구(합)
7  01  FRI     0  1111000000  1111000000   0  20   EE         41.5      242
---------------------------------------------------------------------------------------
2
     대상연월 요일  도착시간  출발 시군구 코드  도착 시군구 코드 성별  나이 이동유형  평균 이동 시간(분) 이동인구(합)
0  202302  일     0      11010      11010  F   0   EH          211   27.44
   유동월   요일  도착시간   출발 시군구 코드   도착 시군구 코드  성별  나이 이동유형  평균 이동 시간(분)  이동인구(합)
10  02  FRI     0  1111000000  1111000000   0  20   EE         34.0      260
---------------------------------------------------------------------------------------
3
     대상연월 요일  도착시간  출발 시군구 코드  도착 시군구 코드 성별  나이 이동유형  평균 이동 시간(분) 이동인구(합)
0  202303  일     0      11010      11010  F   0   EE           26   28.07
  유동월   요일  도착시간   출발 시군구 코드   도착 시군구 코드  성별  나이 이동유형  평균 이동 시간(분)  

# 1~9월 데이터 합치기

In [12]:
data = pd.read_csv('./data/preprocessing_data/processing_data_1.csv', encoding = 'utf-8')
data.head()

,유동월,요일,도착시간,출발 시군구 코드,도착 시군구 코드,성별,나이,이동유형,평균 이동 시간(분),이동인구(합)
0,1,FRI,0,1111000000,1111000000,0,20,EE,41.5,242
1,1,FRI,0,1111000000,1111000000,0,20,EH,59.5,258
2,1,FRI,0,1111000000,1111000000,0,20,EW,21.0,22
3,1,FRI,0,1111000000,1111000000,0,20,HE,20.5,41
4,1,FRI,0,1111000000,1111000000,0,20,HH,32.5,81


In [13]:
for i in range(2, 10):
    path = f'./data/preprocessing_data/processing_data_{i}.csv'
    tmp = pd.read_csv(path, encoding = 'utf-8')

    data = pd.concat([data, tmp], ignore_index=True)
    print(data.shape)

(10426276, 10)
(15779731, 10)
(21127748, 10)
(26548826, 10)
(31924809, 10)
(37309677, 10)
(42665339, 10)
(48042460, 10)


In [14]:
data.head()

,유동월,요일,도착시간,출발 시군구 코드,도착 시군구 코드,성별,나이,이동유형,평균 이동 시간(분),이동인구(합)
0,1,FRI,0,1111000000,1111000000,0,20,EE,41.5,242
1,1,FRI,0,1111000000,1111000000,0,20,EH,59.5,258
2,1,FRI,0,1111000000,1111000000,0,20,EW,21.0,22
3,1,FRI,0,1111000000,1111000000,0,20,HE,20.5,41
4,1,FRI,0,1111000000,1111000000,0,20,HH,32.5,81


In [15]:
data.tail()

,유동월,요일,도착시간,출발 시군구 코드,도착 시군구 코드,성별,나이,이동유형,평균 이동 시간(분),이동인구(합)
48042455,9,TUE,23,1174000000,1174000000,1,60,HH,44.0,376
48042456,9,TUE,23,1174000000,1174000000,1,60,HW,32.0,115
48042457,9,TUE,23,1174000000,1174000000,1,60,WE,78.5,22
48042458,9,TUE,23,1174000000,1174000000,1,60,WH,45.5,780
48042459,9,TUE,23,1174000000,1174000000,1,60,WW,27.0,74


In [16]:
data.to_csv('./data/preprocessing_data/processing_floating_data.csv', encoding = 'utf-8')